In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import matplotlib.pyplot as plt
import os

fase = '01_LecturaDatos'

In [2]:
with open('gen_config.json', 'r') as file:
    gen_config =json.load(file)

In [3]:
folder = gen_config['folder']

path_group = gen_config['path_group']
path_prod_stats = gen_config['path_prod_stats']
path_prod_data = gen_config['path_prod_data']
path_stock_data = gen_config['path_stock_data']

print(f"{'COMIENZA':-^100}")
print(f"{fase:-^100}")

----------------------------------------------COMIENZA----------------------------------------------
------------------------------------------01_LecturaDatos-------------------------------------------


In [4]:
df = pd.read_csv('../sell-in.txt',sep='\t')
prod_a_predecir = pd.read_csv('../productos_a_predecir.txt', sep='\t')
prod_data = pd.read_csv('../tb_productos_descripcion.txt', sep='\t')
stock_data = pd.read_csv('../tb_stocks.txt', sep='\t')

In [5]:
df = df[df['product_id'].isin(prod_a_predecir['product_id'])]

In [34]:
df_grouped = df.groupby(['periodo', 'product_id', 'customer_id'])[['tn']].sum().reset_index()

In [43]:
prod_stats = df_grouped.groupby(['product_id','customer_id']).apply(lambda group: pd.Series({
    'primer_periodo': group['periodo'].min(),
    'ultimo_periodo': group['periodo'].max(),
    'values': group['tn'].count(),
    'total_tn': group['tn'].sum(),
    'min_tn': group['tn'].min(),
    'average_tn': group['tn'].mean(),
    'median_tn': group['tn'].median(),
    'std_dev_tn': group['tn'].std(),
    'iqr_tn': (group['tn'].quantile(0.75) - group['tn'].quantile(0.25)),
    'max_tn': group['tn'].max()
}))
prod_stats['primer_periodo'] = prod_stats['primer_periodo'].astype(int)
prod_stats['primer_periodo'] = pd.to_datetime(prod_stats['primer_periodo'].astype(str), format='%Y%m')

prod_stats['ultimo_periodo'] = prod_stats['ultimo_periodo'].astype(int)
prod_stats['ultimo_periodo'] = pd.to_datetime(prod_stats['ultimo_periodo'].astype(str), format='%Y%m')

prod_stats.reset_index(inplace=True)
print(f"{'Tamaño prod_stats':.<25}: {prod_stats.shape}")

In [45]:
if not os.path.exists(folder):
    os.makedirs(folder)

df_grouped.to_parquet(f'{folder}/{path_group}', index=False)
prod_stats.to_parquet(f'{folder}/{path_prod_stats}', index=False)
prod_data.to_parquet(f'{folder}/{path_prod_data}', index=False)
stock_data.to_parquet(f'{folder}/{path_stock_data}', index=False)

In [46]:
print(f"{fase:-^100}")
print(f"{'FINALIZA':-^100}\n\n\n")

------------------------------------------01_LecturaDatos-------------------------------------------
----------------------------------------------FINALIZA----------------------------------------------



